In [1]:
import pandas as pd
import numpy as np

from numpy.linalg import lstsq
from sklearn.cluster import KMeans
from matplotlib import pyplot as plt

from numba import njit

In [2]:
G = 3

In [3]:
from linearmodels.datasets import wage_panel
wage_panel_df = wage_panel.load()

wage_panel_df.set_index(['nr', 'year'], inplace=True)
y = wage_panel_df['lwage']
x = wage_panel_df.drop(columns=["occupation", "lwage", "black", "hisp", "educ"])
display(x)
display(y)

exper  hours  married  union  expersq
nr    year                                       
13    1980      1   2672        0      0        1
      1981      2   2320        0      1        4
      1982      3   2940        0      0        9
      1983      4   2960        0      0       16
      1984      5   3071        0      0       25
...           ...    ...      ...    ...      ...
12548 1983      8   2080        1      0       64
      1984      9   2080        1      1       81
      1985     10   2080        1      0      100
      1986     11   2080        1      1      121
      1987     12   3380        1      1      144

[4360 rows x 5 columns]

nr     year
13     1980    1.197540
       1981    1.853060
       1982    1.344462
       1983    1.433213
       1984    1.568125
                 ...   
12548  1983    1.591879
       1984    1.212543
       1985    1.765962
       1986    1.745894
       1987    1.466543
Name: lwage, Length: 4360, dtype: float64

In [4]:
N = x.index.get_level_values(0).nunique()
T = x.index.get_level_values(1).nunique()
K = x.shape[1]
N, T, K

(545, 8, 5)

In [5]:
x = x.values.reshape(N, T, K)
y = y.values.reshape(N, T, 1)
x, y

(array([[[   1, 2672,    0,    0,    1],
         [   2, 2320,    0,    1,    4],
         [   3, 2940,    0,    0,    9],
         ...,
         [   6, 2864,    0,    0,   36],
         [   7, 2994,    0,    0,   49],
         [   8, 2640,    0,    0,   64]],
 
        [[   4, 2484,    0,    0,   16],
         [   5, 2804,    0,    0,   25],
         [   6, 2530,    0,    0,   36],
         ...,
         [   9, 2164,    0,    0,   81],
         [  10, 2749,    0,    0,  100],
         [  11, 2476,    0,    0,  121]],
 
        [[   4, 2332,    1,    0,   16],
         [   5, 2116,    1,    0,   25],
         [   6, 2500,    1,    0,   36],
         ...,
         [   9, 2340,    1,    0,   81],
         [  10, 2340,    1,    0,  100],
         [  11, 2340,    1,    0,  121]],
 
        ...,
 
        [[   4, 2008,    1,    0,   16],
         [   5, 3190,    0,    0,   25],
         [   6, 2584,    0,    0,   36],
         ...,
         [   9, 2290,    1,    0,   81],
         [  10, 31

In [6]:
# # Compute theta
# # theta, ssr, rank, singular = lstsq(x_stack, , rcond=None)
# iterations = 10000
# objective_value = np.inf
# tolerance = 1e-8
# G = 3

# x_stack = x.reshape(-1, K)
# y_stack = y.reshape(-1, 1)

# num_start_vars = x.shape[1] + G
# starting_vals = np.random.choice(N, num_start_vars, replace=False)
# x_stack_start = x[starting_vals].reshape(-1, K)
# y_stack_start = y[starting_vals].reshape(-1, 1)

# # NOTE in the paper this should be done on just a subset of the data for initialization
# theta, ssr, rank, singular = lstsq(x_stack_start, y_stack_start, rcond=None)
# residuals = y_stack - x_stack @ theta
# # residuals.reshape(N, T)
# theta

# random_draws = np.random.choice(N, size=G, replace=False)
# alpha = np.squeeze(y[random_draws] - x[random_draws, :, :] @ theta)
# alpha

# res = np.squeeze(y - x @ theta)
# euclidean_distance_between_grouping = ((res[None, :, :] - alpha[:, None, :]) ** 2).sum(axis=2)
# # FIXME some code should check if all values are included, and should create a new grouping if not
# g = np.argmin(euclidean_distance_between_grouping, axis=0)  # Closest group
# g

# res = np.squeeze(y - x @ theta)

# counts = np.bincount(g)[:, None]  # (G, 1) — number of elements in each group
# sums = np.zeros((G, res.shape[1]))  # (G, K) — sum of residuals per group
# np.add.at(sums, g, res)  # sums[i] += res[j] for all j where g[j] == i
# alpha = sums / counts  # mean = sum / count
# alpha

# obj_val_array = np.zeros(iterations)

# for i in range(iterations):
#     # Compute alpha
#     counts = np.bincount(g)[:, None]  # (G, 1) — number of elements in each group
#     sums = np.zeros((G, res.shape[1]))  # (G, K) — sum of residuals per group
#     np.add.at(sums, g, res)  # sums[i] += res[j] for all j where g[j] == i
#     alpha = sums / counts  # mean = sum / count

#     # Compute theta
#     theta, ssr, rank, singular = lstsq(x_stack, y_stack - alpha[g].reshape(-1, 1), rcond=None) # TODO check if this makes sense

#     # Compute groupings
#     res = np.squeeze(y - x @ theta)
#     euclidean_distance_between_grouping = ((res[None, :, :] - alpha[:, None, :]) ** 2).sum(axis=2)
#     # FIXME some code should check if all values are included, and should create a new grouping if not
#     g = np.argmin(euclidean_distance_between_grouping, axis=0)  # Closest group

#     # Check for convergence
#     new_objective_value = ((res - alpha[g])**2).sum()
#     if abs(objective_value - new_objective_value) < tolerance:
#         break
#     objective_value = new_objective_value

#     obj_val_array[i] = objective_value

# display(i)
# display(objective_value)
# display(g)
# display(alpha)
# display(theta)
# # Recompute alpha

In [ ]:
# @njit
def _get_starting_values(y, x, G: int, N: int, K: int):
    """Generates the starting values of theta"""
    num_start_vars: int = x.shape[1] + G # FIXME I believe that shape is slow in Cython
    random_draws_theta = np.random.choice(N, num_start_vars, replace=False)
    x_stack_start = x[random_draws_theta].reshape(-1, K)
    y_stack_start = y[random_draws_theta].reshape(-1, 1)

    # FIXME some errors may arise, maybe add some checks
    theta_init = lstsq(x_stack_start, y_stack_start, rcond=None)[0]

    random_draws_alpha = np.random.choice(N, size=G, replace=False)
    alpha_init = np.squeeze(y[random_draws_alpha] - x[random_draws_alpha, :, :] @ theta_init)

    return theta_init, alpha_init

@njit
def _compute_groupings(res, alpha):
    """Computes the groupings based on the residuals and alpha"""
    euclidean_distance_between_grouping = ((res[None, :, :] - alpha[:, None, :]) ** 2).sum(axis=2)
    g = np.argmin(euclidean_distance_between_grouping, axis=0)  # Closest group
    return g

# @njit
def _compute_alpha(res, g, G):
    """Computes the alpha values based on the residuals and groupings"""
    counts = np.bincount(g, minlength=G)[:, None]  # (G, 1) — number of elements in each group
    sums = np.zeros((G, res.shape[1]))  # (G, K) — sum of residuals per group
    np.add.at(sums, g, res)  # sums[i] += res[j] for all j where g[j] == i
    alpha = sums / counts  # mean = sum / count
    return alpha

# @njit
def _compute_theta(x, y, alpha, g):
    """Computes the theta values based on the x, y, alpha and groupings"""
    # FIXME check if this makes sense
    K = x.shape[2] # FIXME I believe that shape is slow in Cython
    theta = lstsq(x.reshape(-1, K), y.reshape(-1, 1) - alpha[g].reshape(-1, 1), rcond=None)[0]
    return theta

# @njit
def _compute_residuals(y, x, theta):
    """Computes the residuals based on y, x and theta"""
    res = np.squeeze(y - x @ theta)
    return res

@njit
def _compute_objective_value(res, alpha, g):
    """Computes the objective value based on the residuals, alpha and groupings"""
    objective_value = ((res - alpha[g])**2).sum()
    return objective_value

def _reorder_groups(g, alpha):
    """Reorders the groups based on the first value of alpha"""
    # FIXME this is not the best way to do this
    # But it works for now
    mapping = np.argsort(alpha[:, 0])
    ordered_g = np.argsort(mapping)[g]
    ordered_alpha = alpha[mapping]
    return ordered_g, ordered_alpha


def _grouped_fixed_effects_iteration(y, x, G: int, N: int, K: int, max_iter = 10000, tol=1e-8):
    # FIXME possibly create some sort of array that stores these values
    # Could be used for debugging
    theta, alpha = _get_starting_values(y, x, G, N, K)
    res = _compute_residuals(y, x, theta)
    g = _compute_groupings(res, alpha)

    objective_value = np.inf

    iterations_used = 0
    for i in range(max_iter):
        alpha = _compute_alpha(res, g, G)
        theta = _compute_theta(x, y, alpha, g)
        res = _compute_residuals(y, x, theta)
        alpha = _compute_alpha(res, g, G)
        g = _compute_groupings(res, alpha)
        new_objective_value = _compute_objective_value(res, alpha, g)

        if abs(objective_value - new_objective_value) < tol:
            iterations_used = i
            objective_value = new_objective_value
            break

        objective_value = new_objective_value

    return theta, alpha, g, iterations_used, objective_value

def _compute_eta(y_bar, x_bar, theta):
    """Computes the eta values based on y_bar, x_bar and theta"""
    eta = np.squeeze(y_bar - x_bar @ theta)
    return eta

# FIXME not used right now but still neccesary
def _compute_statistics(objective_value, N, T, K, G):
    """Computes the statistics based on the objective value, N, T, K and G"""
    # FIXME this is not the best way to do this
    # But it works for now
    sigma_squared = 1 / (N * T - G * T - N - K) * objective_value
    BIC = 1/(N*T) * objective_value + sigma_squared * (G * T + N + K) / (N * T)
    return sigma_squared, BIC


def grouped_fixed_effects(y, x, G, max_iter=10000, tol=1e-8, gfe_iterations=20, unit_specific_effects=False):
    """
    Computes the grouped fixed effects using the algorithm described in the paper.
    """
    # FIXME not really required if unit_specific_effects is False
    # But this seems like the easiest implementation
    x_bar = np.mean(x, axis=1, keepdims=True)
    y_bar = np.mean(y, axis=1, keepdims=True)

    best_theta = None
    best_alpha = None
    best_g = None
    best_iterations_used = None
    best_objective_value = np.inf

    N = x.shape[0]
    K = x.shape[2]

    # FIXME this code does not drop constant binary variables by itself
    # The input class should be able to do this
    if unit_specific_effects:
        # Demean x and y
        x = x - x_bar
        y = y - y_bar

    for _ in range(gfe_iterations):
        theta, alpha, g, iterations_used, objective_value = _grouped_fixed_effects_iteration(
            y,
            x,
            G,
            N,
            K,
            max_iter,
            tol
        )

        if objective_value < best_objective_value:
            best_objective_value = objective_value
            best_theta = theta
            best_g, best_alpha = _reorder_groups(g, alpha)
            # best_g, best_alpha = g, alpha
            best_iterations_used = iterations_used

    if unit_specific_effects:
        eta = _compute_eta(y_bar, x_bar, best_theta)
        return best_theta, best_alpha, best_g, eta, best_iterations_used, best_objective_value

    # NOTE None is for lack of unit specific effects
    return best_theta, best_alpha, best_g, None, best_iterations_used, best_objective_value

grouped_fixed_effects(y, x, 3, max_iter=10000, tol=1e-8, gfe_iterations=100, unit_specific_effects=True)[1:3]

(array([[-1.27308016,  0.14713952,  0.18722496,  0.12929793,  0.18195473,
          0.16824806,  0.23583886,  0.2233761 ],
        [-0.20940534, -0.32104309, -0.16929891, -0.045043  ,  0.11026162,
          0.16522911,  0.23434463,  0.23495499],
        [ 0.13984353,  0.12236095,  0.03127673, -0.0252122 , -0.06879357,
         -0.08669119, -0.08572159, -0.02706265]]),
 array([2, 2, 1, 2, 2, 0, 2, 2, 1, 1, 1, 2, 1, 2, 1, 2, 2, 1, 2, 2, 2, 2,
        1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 0, 0, 2, 2, 0, 1, 2,
        2, 2, 1, 2, 1, 2, 1, 2, 1, 1, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 1, 2,
        2, 1, 2, 2, 0, 1, 1, 1, 2, 2, 1, 2, 2, 0, 2, 2, 1, 2, 1, 2, 1, 2,
        1, 2, 2, 1, 2, 2, 1, 2, 0, 0, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 1, 1,
        2, 2, 2, 0, 2, 2, 2, 1, 1, 2, 2, 1, 1, 1, 2, 2, 2, 1, 1, 2, 1, 2,
        2, 1, 1, 2, 2, 2, 1, 2, 1, 1, 2, 2, 1, 2, 1, 2, 2, 2, 1, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 0, 2, 2, 1, 1, 1, 1, 2, 2, 2, 1, 2, 1, 2, 2, 2,
        1, 1, 1, 2, 2, 2, 2, 1, 2, 2,

In [57]:
test = np.array([[0, 1, 2], [3, 4, 5]])
test[[1, 0]]

array([[3, 4, 5],
       [0, 1, 2]])

In [8]:
grouped_fixed_effects(y, x, 3, max_iter=10000, tol=1e-6, gfe_iterations=100)

array([[4.500000e+00, 2.807625e+03, 0.000000e+00, 1.250000e-01,
        2.550000e+01]])

(545, 1, 1)

(array([[ 4.56531735e-02],
        [-8.76268692e-05],
        [ 1.04546615e-01],
        [ 6.09680537e-02],
        [-5.46261795e-03]]),
 array([[1.49548037, 1.56936038, 1.61280633, 1.66662319, 1.78230364,
         1.87430794, 1.98308263, 2.11751816],
        [1.02439658, 1.10644043, 1.16106896, 1.23158154, 1.2985915 ,
         1.36475815, 1.51461777, 1.65992997],
        [1.81953868, 2.01490575, 2.10068919, 2.16790666, 2.26021116,
         2.35212556, 2.41629442, 2.49623128]]),
 array([1, 0, 2, 0, 2, 0, 2, 1, 0, 1, 1, 2, 2, 2, 2, 0, 0, 2, 2, 0, 0, 0,
        2, 2, 1, 0, 2, 0, 1, 2, 0, 2, 2, 1, 2, 0, 0, 0, 2, 0, 1, 1, 0, 0,
        2, 0, 1, 2, 1, 2, 0, 1, 2, 1, 2, 1, 1, 2, 2, 0, 1, 2, 1, 2, 1, 1,
        1, 0, 1, 2, 2, 0, 2, 0, 0, 1, 2, 0, 0, 2, 2, 0, 0, 0, 1, 0, 1, 1,
        0, 0, 1, 0, 0, 0, 0, 2, 0, 2, 0, 2, 2, 0, 1, 1, 2, 1, 0, 1, 0, 0,
        0, 0, 2, 1, 0, 0, 0, 1, 2, 2, 0, 0, 2, 1, 1, 2, 1, 0, 0, 0, 2, 1,
        1, 1, 2, 2, 0, 0, 0, 1, 1, 0, 2, 1, 0, 0, 1, 2, 0, 1, 2, 1, 2, 1

In [9]:
import bonhomme_manresa

ModuleNotFoundError: No module named 'bonhomme_manresa'